<a href="https://colab.research.google.com/github/gonzalezhomar/Notebooks/blob/main/RutasGTO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openrouteservice

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import folium
from folium.plugins import BeautifyIcon
import pandas as pd
import openrouteservice as ors

In [6]:
oficinas = pd.read_csv('oficinas.csv', encoding= 'unicode_escape', parse_dates=["Open_From", "Open_To"])
oficinas.dtypes

Oficina                                                                  object
Titular                                                                  object
Días de atención Otros Trámites                                          object
Días de atención Trámites Vehiculares                                    object
Open_From                                                        datetime64[ns]
Open_To                                                          datetime64[ns]
Domicilio                                                                object
Teléfono para citas vehiculares                                          object
Teléfono para Asesoría y Orientación temas fiscales estatales            object
Recepción de llamadas para Citas                                         object
Horario Atención sin citas para tramites vehiculares                     object
Latitude                                                                float64
Longitude                               

In [7]:
# Mapeando las oficinas
m = folium.Map(location=[20.972658157717838, -101.05460389352301], tiles='cartodbpositron', zoom_start=8.5)
for location in oficinas.itertuples():
    tooltip = folium.map.Tooltip("<h4> Oficina <b>{}</b></p>".format(location.Oficina))
    folium.Marker(
        location=[location.Latitude, location.Longitude],
        tooltip=tooltip,
        icon=BeautifyIcon(
            icon_shape='marker'#,
            # number=int(location.id),
            # spin=True,
            # text_color='red',
            # background_color="#FFF",
            # inner_icon_style="font-size:12px;padding-top:-5px;"
        )
    ).add_to(m)

# Punto de partida
depot = [21.0018469, -101.5082909]
folium.Marker(
    location=depot,
    icon=folium.Icon(color="green", icon="bus", prefix='fa'),
    setZIndexOffset=1000
).add_to(m)
m

In [9]:
# Se definen los vehículos
vehicles = list()
for idx in range(3):
    vehicles.append(
        ors.optimization.Vehicle(
            id=idx,
            start=list(reversed(depot)),
            end=list(reversed(depot)),
            capacity=[300],
            time_window=[1553241600, 1553284800]
        )
    )

# Next define the delivery stations
deliveries = list()
for delivery in oficinas.itertuples():
    deliveries.append(
        ors.optimization.Job(
            id=delivery.Index,
            location=[delivery.Longitude, delivery.Latitude],
            service=1200,  # 20 minutes at each site
            # amount=[delivery.Needed_Amount],
            time_windows=[[
                int(delivery.Open_From.timestamp()),  # VROOM expects UNIX timestamp
                int(delivery.Open_To.timestamp())
            ]]
        )
    )

In [10]:
ors_client = ors.Client(key='5b3ce3597851110001cf62480251f7e30f684173a783014c4c85c237')
result = ors_client.optimization(
    jobs=deliveries,
    vehicles=vehicles,
    geometry=True
)

In [11]:
# Add the output to the map
for color, route in zip(['green', 'red', 'blue'], result['routes']):
    decoded = ors.convert.decode_polyline(route['geometry'])  # Route geometry is encoded
    gj = folium.GeoJson(
        name='Vehicle {}'.format(route['vehicle']),
        data={"type": "FeatureCollection", "features": [{"type": "Feature",
                                                         "geometry": decoded,
                                                         "properties": {"color": color}
                                                         }]},
        style_function=lambda x: {"color": x['properties']['color']}
    )
    gj.add_child(folium.Tooltip(
        """<h4>Vehículo {vehicle}</h4>
        <b>Distancia</b> {distance} m <br>
        <b>Duración</b> {duration} secs
        """.format(**route)
    ))
    gj.add_to(m)

folium.LayerControl().add_to(m)
m